# Homework 5
In this homework you will improve your convolutional network to overfit less on supertux.

Development notes: 

1) If you are doing your homework in a Jupyter/iPython notebook you may need to 'Restart & Clear Output' after making a change and re-running a cell.  TensorFlow will not allow you to create multiple variables with the same name, which is what you are doing when you run a cell that creates a variable twice.<br/><br/>
2) Be careful with your calls to global_variables_initializer(). If you call it after training one network it will re-initialize your variables erasing your training.  In general, double check the outputs of your model after all training and before turning your model in. Ending a session will discard all your variable values.

## Part 0: Setup

In [1]:
import tensorflow as tf
import numpy as np
import util

# Load the data we are giving you
def load(filename, W=64, H=64):
    data = np.fromfile(filename, dtype=np.uint8).reshape((-1, W*H*3+1))
    images, labels = data[:, :-1].reshape((-1,H,W,3)), data[:, -1]
    return images, labels

image_data, label_data = load('tux_train.dat')

print('Input shape: ' + str(image_data.shape))
print('Labels shape: ' + str(label_data.shape))

num_classes = 6

Input shape: (12257, 64, 64, 3)
Labels shape: (12257,)


## Part 1: Define your convnet

Make sure the total number of parameters is less than 100,000.

In [2]:
# Lets clear the tensorflow graph, so that you don't have to restart the notebook every time you change the network
tf.reset_default_graph()

# Set up your input placeholder
inputs = tf.placeholder(tf.float32, (None,64,64,3))

# Step 1: Augment the training data (try the following, not all might improve the performance)
#  * mirror the image
#  * color augmentations (keep the values to small ranges first then try to expand):
#    - brightness
#    - hue
#    - saturation
#    - contrast

def data_augmentation(I):
    # TODO: Put your data augmentation here
#     I= tf.image.random_brightness(I,20)
#     I= tf.image.random_saturation(I,0,50)
    I= tf.image.random_flip_left_right(I)
#     I= tf.image.random_flip_up_down(I)
#     I= tf.image.random_hue(I,0.2)
#     I= tf.image.random_contrast(I,0,0.5)
    return I

# map_fn applies data_augmentation independently for each image in the batch, since we are not croping let's apply the augmentation before whitening, it does make evaluation easier
aug_input = tf.map_fn(data_augmentation, inputs)

# During evaluation we don't want data augmentation
eval_inputs = tf.identity(aug_input, name='inputs')

# Whenever you deal with image data it's important to mean center it first and subtract the standard deviation
white_inputs = (eval_inputs - 100.) / 72.


# Set up your label placeholders
labels = tf.placeholder(tf.int64, (None), name='labels')
lrate= tf.placeholder(tf.float32,name='learning_rate')
outputs = []
losses = []


In [3]:

# Step 4: Define multiple models in your ensemble. You should train an ensemble of 5 models.
# Let's put all variables in a scope, this makes training ensembles easier. Make sure each model in your ensemble has it's own scope and produces an output and loss
with tf.name_scope('model1'), tf.variable_scope('model1'):
    # Step 2: define the compute graph of your CNN here (use your solution to HW4 here)
    #   Add weight regularization (l2-loss)
    
    conv1 = tf.contrib.layers.conv2d(inputs=white_inputs,num_outputs=19,kernel_size=[5, 5],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv1') # put in scope everywhere
    print(conv1)
    conv1_bn= tf.layers.batch_normalization(conv1,training=True)
   
    conv2 = tf.contrib.layers.conv2d(inputs=conv1_bn,num_outputs=30,kernel_size=[5, 5],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv2') # put in scope everywhere
    print(conv2)
    conv2_bn= tf.layers.batch_normalization(conv2,training=True)
    
    
    conv3 = tf.contrib.layers.conv2d(inputs=conv2_bn,num_outputs=50,kernel_size=[5,5],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv3') # put in scope everywhere
    print(conv3)
    conv3_bn= tf.layers.batch_normalization(conv3,training=True)

    
    conv4 = tf.contrib.layers.conv2d(inputs=conv3_bn,num_outputs=100,kernel_size=[3, 3],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv4') # put in scope everywhere
    print(conv4)
    conv4_bn= tf.layers.batch_normalization(conv4,training=True)
    pool4 = tf.contrib.layers.max_pool2d(inputs=conv4_bn, kernel_size=[2,2], stride=2, scope='pool4')
    print(pool4)
    
    conv5 = tf.contrib.layers.conv2d(inputs=pool4,num_outputs=6,kernel_size=[1,1],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv5') # put in scope everywhere
    print(conv5)
    
    
    h= tf.identity(conv5, name='conv_out')
        
    #put it all together
    h = tf.contrib.layers.flatten(h)
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=labels))
    outputs.append(h)
    losses.append(loss)


Tensor("model1/model1/conv1/Relu:0", shape=(?, 32, 32, 19), dtype=float32)
Tensor("model1/model1/conv2/Relu:0", shape=(?, 16, 16, 30), dtype=float32)
Tensor("model1/model1/conv3/Relu:0", shape=(?, 8, 8, 50), dtype=float32)
Tensor("model1/model1/conv4/Relu:0", shape=(?, 4, 4, 100), dtype=float32)
Tensor("model1/model1/pool4/MaxPool:0", shape=(?, 2, 2, 100), dtype=float32)
Tensor("model1/model1/conv5/Relu:0", shape=(?, 1, 1, 6), dtype=float32)


In [4]:

# Step 4: Define multiple models in your ensemble. You should train an ensemble of 5 models.
# Let's put all variables in a scope, this makes training ensembles easier. Make sure each model in your ensemble has it's own scope and produces an output and loss
with tf.name_scope('model2'), tf.variable_scope('model2'):
    # Step 2: define the compute graph of your CNN here (use your solution to HW4 here)
    #   Add weight regularization (l2-loss)
    
    conv1 = tf.contrib.layers.conv2d(inputs=white_inputs,num_outputs=19,kernel_size=[5, 5],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv1') # put in scope everywhere
    print(conv1)
    conv1_bn= tf.layers.batch_normalization(conv1,training=True)
   
    conv2 = tf.contrib.layers.conv2d(inputs=conv1_bn,num_outputs=30,kernel_size=[5, 5],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv2') # put in scope everywhere
    print(conv2)
    conv2_bn= tf.layers.batch_normalization(conv2,training=True)
    
    
    conv3 = tf.contrib.layers.conv2d(inputs=conv2_bn,num_outputs=50,kernel_size=[5,5],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv3') # put in scope everywhere
    print(conv3)
    conv3_bn= tf.layers.batch_normalization(conv3,training=True)

    
    conv4 = tf.contrib.layers.conv2d(inputs=conv3_bn,num_outputs=100,kernel_size=[3, 3],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv4') # put in scope everywhere
    print(conv4)
#     drop4= tf.contrib.layers.dropout(inputs=conv4,keep_prob=0.5)
    conv4_bn= tf.layers.batch_normalization(conv4,training=True)
    pool4 = tf.contrib.layers.max_pool2d(inputs=conv4_bn, kernel_size=[2,2], stride=2, scope='pool4')
    print(pool4)
    
    conv5 = tf.contrib.layers.conv2d(inputs=pool4,num_outputs=6,kernel_size=[1,1],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv5') # put in scope everywhere
    print(conv5)
    
    
    h= tf.identity(conv5, name='conv_out')
        
    #put it all together
    h = tf.contrib.layers.flatten(h)
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=labels))
    outputs.append(h)
    losses.append(loss)

Tensor("model2/model2/conv1/Relu:0", shape=(?, 32, 32, 19), dtype=float32)
Tensor("model2/model2/conv2/Relu:0", shape=(?, 16, 16, 30), dtype=float32)
Tensor("model2/model2/conv3/Relu:0", shape=(?, 8, 8, 50), dtype=float32)
Tensor("model2/model2/conv4/Relu:0", shape=(?, 4, 4, 100), dtype=float32)
Tensor("model2/model2/pool4/MaxPool:0", shape=(?, 2, 2, 100), dtype=float32)
Tensor("model2/model2/conv5/Relu:0", shape=(?, 1, 1, 6), dtype=float32)


In [5]:

# Step 4: Define multiple models in your ensemble. You should train an ensemble of 5 models.
# Let's put all variables in a scope, this makes training ensembles easier. Make sure each model in your ensemble has it's own scope and produces an output and loss
with tf.name_scope('model3'), tf.variable_scope('model3'):
    # Step 2: define the compute graph of your CNN here (use your solution to HW4 here)
    #   Add weight regularization (l2-loss)
    
    conv1 = tf.contrib.layers.conv2d(inputs=white_inputs,num_outputs=19,kernel_size=[5, 5],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv1') # put in scope everywhere
    print(conv1)
    conv1_bn= tf.layers.batch_normalization(conv1,training=True)
   
    conv2 = tf.contrib.layers.conv2d(inputs=conv1_bn,num_outputs=30,kernel_size=[5, 5],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv2') # put in scope everywhere
    print(conv2)
    conv2_bn= tf.layers.batch_normalization(conv2,training=True)
    
    
    conv3 = tf.contrib.layers.conv2d(inputs=conv2_bn,num_outputs=50,kernel_size=[5,5],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv3') # put in scope everywhere
    print(conv3)
    conv3_bn= tf.layers.batch_normalization(conv3,training=True)

    
    conv4 = tf.contrib.layers.conv2d(inputs=conv3_bn,num_outputs=100,kernel_size=[3, 3],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv4') # put in scope everywhere
    print(conv4)
#     drop4= tf.contrib.layers.dropout(inputs=conv4,keep_prob=0.5)
    conv4_bn= tf.layers.batch_normalization(conv4,training=True)
    pool4 = tf.contrib.layers.max_pool2d(inputs=conv4_bn, kernel_size=[2,2], stride=2, scope='pool4')
    print(pool4)
    
    conv5 = tf.contrib.layers.conv2d(inputs=pool4,num_outputs=6,kernel_size=[1,1],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv5') # put in scope everywhere
    print(conv5)
    
    
    h= tf.identity(conv5, name='conv_out')
        
    #put it all together
    h = tf.contrib.layers.flatten(h)
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=labels))
    outputs.append(h)
    losses.append(loss)

Tensor("model3/model3/conv1/Relu:0", shape=(?, 32, 32, 19), dtype=float32)
Tensor("model3/model3/conv2/Relu:0", shape=(?, 16, 16, 30), dtype=float32)
Tensor("model3/model3/conv3/Relu:0", shape=(?, 8, 8, 50), dtype=float32)
Tensor("model3/model3/conv4/Relu:0", shape=(?, 4, 4, 100), dtype=float32)
Tensor("model3/model3/pool4/MaxPool:0", shape=(?, 2, 2, 100), dtype=float32)
Tensor("model3/model3/conv5/Relu:0", shape=(?, 1, 1, 6), dtype=float32)


In [6]:

# Step 4: Define multiple models in your ensemble. You should train an ensemble of 5 models.
# Let's put all variables in a scope, this makes training ensembles easier. Make sure each model in your ensemble has it's own scope and produces an output and loss
with tf.name_scope('model4'), tf.variable_scope('model4'):
    # Step 2: define the compute graph of your CNN here (use your solution to HW4 here)
    #   Add weight regularization (l2-loss)
    
    conv1 = tf.contrib.layers.conv2d(inputs=white_inputs,num_outputs=19,kernel_size=[5, 5],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv1') # put in scope everywhere
    print(conv1)
    conv1_bn= tf.layers.batch_normalization(conv1,training=True)
   
    conv2 = tf.contrib.layers.conv2d(inputs=conv1_bn,num_outputs=30,kernel_size=[5, 5],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv2') # put in scope everywhere
    print(conv2)
    conv2_bn= tf.layers.batch_normalization(conv2,training=True)
    
    
    conv3 = tf.contrib.layers.conv2d(inputs=conv2_bn,num_outputs=50,kernel_size=[5,5],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv3') # put in scope everywhere
    print(conv3)
    conv3_bn= tf.layers.batch_normalization(conv3,training=True)

    
    conv4 = tf.contrib.layers.conv2d(inputs=conv3_bn,num_outputs=100,kernel_size=[3, 3],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv4') # put in scope everywhere
    print(conv4)
#     drop4= tf.contrib.layers.dropout(inputs=conv4,keep_prob=0.5)
    conv4_bn= tf.layers.batch_normalization(conv4,training=True)
    pool4 = tf.contrib.layers.max_pool2d(inputs=conv4_bn, kernel_size=[2,2], stride=2, scope='pool4')
    print(pool4)
    
    conv5 = tf.contrib.layers.conv2d(inputs=pool4,num_outputs=6,kernel_size=[1,1],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv5') # put in scope everywhere
    print(conv5)
    
    
    h= tf.identity(conv5, name='conv_out')
        
    #put it all together
    h = tf.contrib.layers.flatten(h)
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=labels))
    outputs.append(h)
    losses.append(loss)


Tensor("model4/model4/conv1/Relu:0", shape=(?, 32, 32, 19), dtype=float32)
Tensor("model4/model4/conv2/Relu:0", shape=(?, 16, 16, 30), dtype=float32)
Tensor("model4/model4/conv3/Relu:0", shape=(?, 8, 8, 50), dtype=float32)
Tensor("model4/model4/conv4/Relu:0", shape=(?, 4, 4, 100), dtype=float32)
Tensor("model4/model4/pool4/MaxPool:0", shape=(?, 2, 2, 100), dtype=float32)
Tensor("model4/model4/conv5/Relu:0", shape=(?, 1, 1, 6), dtype=float32)


In [7]:

# Step 4: Define multiple models in your ensemble. You should train an ensemble of 5 models.
# Let's put all variables in a scope, this makes training ensembles easier. Make sure each model in your ensemble has it's own scope and produces an output and loss
with tf.name_scope('model5'), tf.variable_scope('model5'):
    # Step 2: define the compute graph of your CNN here (use your solution to HW4 here)
    #   Add weight regularization (l2-loss)
    
    conv1 = tf.contrib.layers.conv2d(inputs=white_inputs,num_outputs=19,kernel_size=[5, 5],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv1') # put in scope everywhere
    print(conv1)
    conv1_bn= tf.layers.batch_normalization(conv1,training=True)
   
    conv2 = tf.contrib.layers.conv2d(inputs=conv1_bn,num_outputs=30,kernel_size=[5, 5],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv2') # put in scope everywhere
    print(conv2)
    conv2_bn= tf.layers.batch_normalization(conv2,training=True)
    
    
    conv3 = tf.contrib.layers.conv2d(inputs=conv2_bn,num_outputs=50,kernel_size=[5,5],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv3') # put in scope everywhere
    print(conv3)
    conv3_bn= tf.layers.batch_normalization(conv3,training=True)

    
    conv4 = tf.contrib.layers.conv2d(inputs=conv3_bn,num_outputs=100,kernel_size=[3, 3],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv4') # put in scope everywhere
    print(conv4)
#     drop4= tf.contrib.layers.dropout(inputs=conv4,keep_prob=0.5)
    conv4_bn= tf.layers.batch_normalization(conv4,training=True)
    pool4 = tf.contrib.layers.max_pool2d(inputs=conv4_bn, kernel_size=[2,2], stride=2, scope='pool4')
    print(pool4)
    
    conv5 = tf.contrib.layers.conv2d(inputs=pool4,num_outputs=6,kernel_size=[1,1],weights_regularizer=tf.nn.l2_loss,
      stride=2,padding="same",scope='conv5') # put in scope everywhere
    print(conv5)
    
    
    h= tf.identity(conv5, name='conv_out')
        
    #put it all together
    h = tf.contrib.layers.flatten(h)
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=labels))
    outputs.append(h)
    losses.append(loss)



Tensor("model5/model5/conv1/Relu:0", shape=(?, 32, 32, 19), dtype=float32)
Tensor("model5/model5/conv2/Relu:0", shape=(?, 16, 16, 30), dtype=float32)
Tensor("model5/model5/conv3/Relu:0", shape=(?, 8, 8, 50), dtype=float32)
Tensor("model5/model5/conv4/Relu:0", shape=(?, 4, 4, 100), dtype=float32)
Tensor("model5/model5/pool4/MaxPool:0", shape=(?, 2, 2, 100), dtype=float32)
Tensor("model5/model5/conv5/Relu:0", shape=(?, 1, 1, 6), dtype=float32)


In [8]:
output = tf.add_n(outputs, name='output')

# Sum up all the losses
loss = tf.add_n(losses)
regularization_loss = tf.losses.get_regularization_loss()
# Let's weight the regularization loss down, otherwise it will hurt the model performance
# You can tune this weight if you wish
total_loss = loss + 1e-6 * regularization_loss

# create an optimizer: Adam might work slightly better (it's a bit faster for Tux)
optimizer = tf.train.AdamOptimizer(lrate, 0.9, 0.999)

# use that optimizer on your loss function
opt = optimizer.minimize(total_loss)
correct = tf.equal(tf.argmax(output, 1), labels)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# You're allowed to use 500k variables this time, 100k per model in your ensemble.
print( "Total number of variables used ", np.sum([v.get_shape().num_elements() for v in tf.trainable_variables()]), '/', 500000 )

Total number of variables used  496890 / 500000


## Part 2: Training

Training might take up to 20 min depending on your architecture.  This time around you should get close to 100% training accuracy.

In [9]:
image_val, label_val = load('tux_val.dat')

# Batch size
BS = 32

# Start a session
sess = tf.Session()

# Set up training
sess.run(tf.global_variables_initializer())

# Train convnet
# Step 3: You should tune the number of epochs to maximize validation accuracy, you can either do this by hand or automate the process.
# Adding an automated way to early stopping, we're also saving the model if it's the best valid accuracy.

for epoch in range(25):
    # Let's shuffle the data every epoch
    np.random.seed(epoch)
    np.random.shuffle(image_data)
    np.random.seed(epoch)
    np.random.shuffle(label_data)
    # Go through the entire dataset once
    accuracy_vals, loss_vals = [], []
    for i in range(0, image_data.shape[0]-BS+1, BS):
        # Train a single batch
        batch_images, batch_labels = image_data[i:i+BS], label_data[i:i+BS]
        accuracy_val, loss_val, _ = sess.run([accuracy, total_loss, opt], feed_dict={inputs: batch_images, labels: batch_labels,lrate:0.0005})
        accuracy_vals.append(accuracy_val)
        loss_vals.append(loss_val)

    val_correct = []
    for i in range(0, image_val.shape[0], BS):
        batch_images, batch_labels = image_val[i:i+BS], label_val[i:i+BS]
        val_correct.extend( sess.run(correct, feed_dict={eval_inputs: batch_images, labels: batch_labels}) )
    print('[%3d] Accuracy: %0.3f  \t  Loss: %0.3f  \t  validation accuracy: %0.3f'%(epoch, np.mean(accuracy_vals), np.mean(loss_vals), np.mean(val_correct)))

[  0] Accuracy: 0.928  	  Loss: 1.824  	  validation accuracy: 0.914
[  1] Accuracy: 0.971  	  Loss: 0.737  	  validation accuracy: 0.931
[  2] Accuracy: 0.982  	  Loss: 0.431  	  validation accuracy: 0.946
[  3] Accuracy: 0.987  	  Loss: 0.328  	  validation accuracy: 0.945
[  4] Accuracy: 0.990  	  Loss: 0.274  	  validation accuracy: 0.945
[  5] Accuracy: 0.992  	  Loss: 0.234  	  validation accuracy: 0.950
[  6] Accuracy: 0.995  	  Loss: 0.185  	  validation accuracy: 0.957
[  7] Accuracy: 0.996  	  Loss: 0.157  	  validation accuracy: 0.963
[  8] Accuracy: 0.996  	  Loss: 0.156  	  validation accuracy: 0.948
[  9] Accuracy: 0.997  	  Loss: 0.135  	  validation accuracy: 0.956
[ 10] Accuracy: 0.997  	  Loss: 0.128  	  validation accuracy: 0.960
[ 11] Accuracy: 0.998  	  Loss: 0.108  	  validation accuracy: 0.965
[ 12] Accuracy: 0.998  	  Loss: 0.088  	  validation accuracy: 0.965
[ 13] Accuracy: 0.998  	  Loss: 0.107  	  validation accuracy: 0.963
[ 14] Accuracy: 0.998  	  Loss: 0.

## Part 3: Evaluation

### Compute the valiation accuracy

In [10]:
image_val, label_val = load('tux_val.dat')

print('Input shape: ' + str(image_val.shape))
print('Labels shape: ' + str(label_val.shape))

val_correct = []
for i in range(0, image_val.shape[0], BS):
    batch_images, batch_labels = image_val[i:i+BS], label_val[i:i+BS]
    val_correct.extend( sess.run(correct, feed_dict={eval_inputs: batch_images, labels: batch_labels}) )
print("ConvNet Validation Accuracy: ", np.mean(val_correct))

Input shape: (3912, 64, 64, 3)
Labels shape: (3912,)
ConvNet Validation Accuracy:  0.966768916155


## Part 4: Save Model
Please note that we also want you to turn in your ipynb for this assignment.  Zip up the ipynb along with the tfg for your submission.

In [11]:
# util.save('assignment5_v3_9716.tfg', session=sess)

### Part 5 (optional): See your model

In [12]:
# Show the current graph
util.show_graph(tf.get_default_graph().as_graph_def())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
